In [1]:
import numpy as np
from numpy import array
import math

import pandas as pd 
from pandas import DataFrame
from pandas import concat
from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier

# keras imports for the dataset and building our neural network
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

# to calculate accuracy
from sklearn.metrics import accuracy_score

#ENSEMBLE LEARNING
from sklearn.tree import DecisionTreeClassifier
from sklearn. ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import tensorflow as tf
import tensorflow.keras.layers as KL

from matplotlib import pyplot

In [2]:
input_data = pd.read_csv (r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/input.csv', index_col = 0)
input_data.index.name = 'Timestamp (seconds)'

In [3]:
output_data = pd.read_csv (r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/output.csv', index_col = 0)
output_data.index.name = 'Timestamp (seconds)'

In [4]:
split_columns = 1

In [5]:
#Modifying the output_data to, putting to Y in array
#0 for normal
#0.5 for drowsy
#1 for aggressive

output_data = np.array(output_data)

Y = list()

for i in range(int(output_data.shape[0]/split_columns)):
    condition = output_data[(i*split_columns)]    
    behavior = []
    if(condition=='DROWSY'):
        behavior = [0]
    elif(condition=='AGGRESSIVE'):
        behavior = [1]
    else:
        behavior = [2]
    
    Y.append(behavior)

Y = array(Y)
print(Y.shape)

(29000, 1)


In [6]:
#min max scaler
X = input_data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
print(X.shape)
print(Y.shape)

(29000, 34)
(29000, 1)


In [7]:
#SPLITTING TRAINING AND TEST DATA METHOD 1
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [8]:
#SPLITTING TRAINING AND TEST DATA METHOD 2
#X_train = X[:(int(X.shape[0] * 0.8)),:,:]
#X_test = X[(int(X.shape[0] * 0.8)):,:,:]
print(X_train.shape)
print(X_test.shape)
#Y_train = Y[:(int(Y.shape[0] * 0.8)),:]
#Y_test = Y[(int(Y.shape[0] * 0.8)):,:]
print(Y_train.shape)
print(Y_test.shape)

(23200, 34)
(5800, 34)
(23200, 1)
(5800, 1)


In [9]:
#MAKING SURE DATA TYPE ARE CORRECT
X_train = np.asarray(X_train).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')

In [10]:
#descision tree
dt = DecisionTreeClassifier()
dt.fit(X_train,Y_train)
print(dt.score(X_test,Y_test))

0.9187931034482759


In [11]:
#Random Forest
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train,Y_train)
rf.score(X_test,Y_test)

<ipython-input-11-1af81321a2f0>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,Y_train)


0.9770689655172414

In [12]:
#Boosting - Ada Boost
adb = AdaBoostClassifier(DecisionTreeClassifier(),n_estimators = 100, learning_rate = 1)
adb.fit(X_train,Y_train)
print(adb.score(X_test,Y_test))

C:\Users\Geraldo Wibowo\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.9189655172413793


In [13]:
#Bagging 
bg = BaggingClassifier(DecisionTreeClassifier(), max_samples= 0.5, max_features = 1.0, n_estimators = 100)
bg.fit(X_train,Y_train)
print(bg.score(X_test,Y_test))

C:\Users\Geraldo Wibowo\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.9570689655172414


In [38]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train,Y_train)
print(lr.score(X_test,Y_test))

C:\Users\Geraldo Wibowo\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.603103448275862


C:\Users\Geraldo Wibowo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
# Voting Classifier - Multiple Model Ensemble 
#lr = LogisticRegression()
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()
bg = BaggingClassifier()
adb = AdaBoostClassifier()
#svm = SVC(kernel = 'poly', degree = 2 )

In [33]:
evc = VotingClassifier( estimators= [('dt',dt),('bg',bg),('rf',rf),('adb',adb)], voting = 'hard')

In [34]:
evc.fit(X_train,Y_train)

C:\Users\Geraldo Wibowo\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('bg', BaggingClassifier()),
                             ('rf', RandomForestClassifier()),
                             ('adb', AdaBoostClassifier())])

In [35]:
print(evc.score(X_test, Y_test))

0.956551724137931
